<a href="https://colab.research.google.com/github/winterForestStump/econometrics/blob/main/assignment_4/ps4_p4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%capture
!pip install linearmodels

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import PooledOLS, PanelOLS, RandomEffects
from linearmodels.panel import compare

In [24]:
data = pd.read_stata('https://github.com/winterForestStump/econometrics/raw/refs/heads/main/assignment_4/grunfeld.dta')

# a)

In [25]:
df = data.copy()

# Prepare panel data
df = df.set_index(['group', 'year'])
df.index.names = ['group', 'year']

# Define variables
df['Investment'] = df['gi']  # Dependent variable
df['Real value'] = df['vf']   # Real value of the firm
df['Capital Stock'] = df['vc'] # Real value of capital stock

# Add constant for pooled OLS
df['Intercept'] = 1

# (a) Pooled OLS
X_pooled = df[['Intercept', 'Real value', 'Capital Stock']]
y_pooled = df['Investment']

pooled_model = PooledOLS(y_pooled, X_pooled)
pooled_results = pooled_model.fit(cov_type='clustered', cluster_entity=True)
print(pooled_results)

                          PooledOLS Estimation Summary                          
Dep. Variable:             Investment   R-squared:                        0.8124
Estimator:                  PooledOLS   R-squared (Between):              0.8357
No. Observations:                 200   R-squared (Within):               0.7385
Date:                Tue, Dec 02 2025   R-squared (Overall):              0.8124
Time:                        06:44:24   Log-likelihood                   -1191.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      426.58
Entities:                          10   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,197)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             57.036
                            

# b)

In [32]:
# (b) Fixed Effects (Within estimator)
df = data.copy()

# Prepare panel data
df = df.set_index(['group', 'year'])
df.index.names = ['group', 'year']

# Define variables
df['Investment'] = df['gi']  # Dependent variable
df['Real value'] = df['vf']   # Real value of the firm
df['Capital Stock'] = df['vc'] # Real value of capital stock

X_pooled = df[['Real value', 'Capital Stock']]
y_pooled = df['Investment']

fe_model = PanelOLS(y_pooled, X_pooled, entity_effects=True, time_effects=True)
fe_results = fe_model.fit(cov_type='clustered', cluster_entity=True)
print(fe_results)


                          PanelOLS Estimation Summary                           
Dep. Variable:             Investment   R-squared:                        0.7201
Estimator:                   PanelOLS   R-squared (Between):              0.7537
No. Observations:                 200   R-squared (Within):               0.7533
Date:                Tue, Dec 02 2025   R-squared (Overall):              0.7536
Time:                        06:47:51   Log-likelihood                   -1056.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      217.44
Entities:                          10   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,169)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             63.066
                            

In [27]:
# Test if FE is better than pooled OLS (F-test for individual effects)
print("\nF-test for significance of fixed effects:")
print(f"F-statistic: {fe_results.f_pooled.stat}")
print(f"P-value: {fe_results.f_pooled.pval}")
if fe_results.f_pooled.pval < 0.05:
    print("Conclusion: Fixed effects are significant → prefer FE over pooled OLS")
else:
    print("Conclusion: Fixed effects not significant → pooled OLS may suffice")
print("\n")


F-test for significance of fixed effects:
F-statistic: 49.17662791363477
P-value: 1.1102230246251565e-16
Conclusion: Fixed effects are significant → prefer FE over pooled OLS




# c)

In [35]:
# (c) Random Effects (FGLS)
df = data.copy()

# Prepare panel data
df = df.set_index(['group', 'year'])
df.index.names = ['group', 'year']

# Define variables
df['Investment'] = df['gi']  # Dependent variable
df['Real value'] = df['vf']   # Real value of the firm
df['Capital Stock'] = df['vc'] # Real value of capital stock

# Add constant for pooled OLS
df['Intercept'] = 1

X_pooled = df[['Intercept', 'Real value', 'Capital Stock']]
y_pooled = df['Investment']

re_model = RandomEffects(y_pooled, X_pooled)
re_results = re_model.fit(cov_type='clustered', cluster_entity=True)
print(re_results)

                        RandomEffects Estimation Summary                        
Dep. Variable:             Investment   R-squared:                        0.7695
Estimator:              RandomEffects   R-squared (Between):              0.8148
No. Observations:                 200   R-squared (Within):               0.7667
Date:                Tue, Dec 02 2025   R-squared (Overall):              0.8033
Time:                        06:48:28   Log-likelihood                   -1075.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      328.84
Entities:                          10   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,197)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             38.764
                            

In [29]:
re_results.variance_decomposition

,Variance Decomposition
Effects,7089.800017
Residual,2784.458053
Percent due to Effects,0.718008


In [37]:
from linearmodels.panel import compare

print(compare({"Pooled": pooled_results, "FE": fe_results, "RE": re_results}))

                            Model Comparison                           
                                Pooled             FE                RE
-----------------------------------------------------------------------
Dep. Variable               Investment     Investment        Investment
Estimator                    PooledOLS       PanelOLS     RandomEffects
No. Observations                   200            200               200
Cov. Est.                    Clustered      Clustered         Clustered
R-squared                       0.8124         0.7201            0.7695
R-Squared (Within)              0.7385         0.7533            0.7667
R-Squared (Between)             0.8357         0.7537            0.8148
R-Squared (Overall)             0.8124         0.7536            0.8033
F-statistic                     426.58         217.44            328.84
P-value (F-stat)                0.0000         0.0000            0.0000
=====================     ============   ============   ========

# d)


In [31]:
# (d) Hausman test
print("(d) Hausman Test")

# Hausman test manually
# H0: Random effects is consistent (prefer RE for efficiency)
# H1: Fixed effects is consistent, RE is not (prefer FE)

# Get coefficients and variance matrices
beta_fe = fe_results.params
beta_re = re_results.params.drop('Intercept')

# Ensure same order
beta_fe = beta_fe.loc[beta_re.index]

# Covariance matrices (use robust if available)
cov_fe = fe_results.cov
cov_re = re_results.cov.loc[beta_re.index, beta_re.index]

# Hausman test statistic
diff = beta_fe - beta_re
cov_diff = cov_fe - cov_re  # Note: Hausman requires cov(beta_FE) - cov(beta_RE) positive definite

# Alternative calculation if cov_diff not positive definite
# Use Moore-Penrose pseudo-inverse
from scipy.linalg import pinv
try:
    # Regular inverse
    cov_diff_inv = np.linalg.inv(cov_diff)
    chi2 = diff @ cov_diff_inv @ diff
    df = len(diff)
    from scipy.stats import chi2 as chi2_dist
    pval = 1 - chi2_dist.cdf(chi2, df)
    print(f"Hausman test statistic (chi2): {chi2:.4f}")
    print(f"Degrees of freedom: {df}")
    print(f"P-value: {pval:.4f}")
except np.linalg.LinAlgError:
    # Use pseudo-inverse
    cov_diff_inv = pinv(cov_diff)
    chi2 = diff @ cov_diff_inv @ diff
    df = np.linalg.matrix_rank(cov_diff)
    from scipy.stats import chi2 as chi2_dist
    pval = 1 - chi2_dist.cdf(chi2, df)
    print(f"Hausman test statistic (chi2, using pseudo-inverse): {chi2:.4f}")
    print(f"Degrees of freedom (rank): {df}")
    print(f"P-value: {pval:.4f}")

# Decision rule
if pval < 0.05:
    print("\nDecision: Reject H0 → FE model is preferred (RE inconsistent)")
    print("Reason: Individual effects (µi) are correlated with regressors")
else:
    print("\nDecision: Cannot reject H0 → RE model is preferred (more efficient)")
    print("Reason: Individual effects appear uncorrelated with regressors")

(d) Hausman Test
Hausman test statistic (chi2): 0.0027
Degrees of freedom: 2
P-value: 0.9987

Decision: Cannot reject H0 → RE model is preferred (more efficient)
Reason: Individual effects appear uncorrelated with regressors
